In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 修正したモデルから卒論時に集計したデータを作成する

1. 表

| ベンチマーク名 | 平均誤差率(%) | コスト比(%) |
|---------|----------|---------|


2. 表

| ベンチマーク名 | 採用割合(最大MAPE(%), 最小MAPE(%)) |
|---------|----------------------------|
|         | モデル(1), モデル(2), ...        |


* 平均誤差率：大規模実行時の関数コール回数との比較
* MAPE：トレーニングデータとの比較

In [ ]:
plt.figure(figsize=(4, 3))
plt.xlabel("使用したプロファイル数")
plt.ylabel("平均誤差率(%)")

In [ ]:
# pd.get_option("display.max_columns")
# pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# ベンチマーク名・関数名・プロセス数・問題サイズを指定することで、その条件での関数コール回数を取得する関数

def returnSpecificData(benchmarkName="cg", functionName=".TAU_application", process=256, benchmarkClass="D"):
    targetRawDF = returnRawDF(Benchmark=benchmarkName, functionName=functionName, benchmarkClass=[
                              benchmarkClass], FixedProcess=process, Processes=[process], FixedBenchmarkClass=benchmarkClass)
    return targetRawDF.iat[0, 0]
# returnSpecificData(benchmarkName="mg", functionName="BUBBLE", process=256, benchmarkClass="B")

In [ ]:
# benchmarksからbt, spを除外する
benchmarks = [benchmark for benchmark in benchmarks if benchmark !=
              'bt' and benchmark != 'sp']
# pandasのDFをprintした時の幅を広げる
pd.set_option('display.width', 100)

In [ ]:
dictTmp = returnDictForPlotPerNumOfUsedData(Benchmark=benchmarks, fix="Class", benchmarkClass=[
    "A", "B", "C", "D"], FixedProcess=64, Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="C")

In [ ]:
pd.options.display.float_format = '{:.4g}'.format

tmpDF = pd.DataFrame()
for benchmark in benchmarks:
    listToLearn = [1, 2, 4, 8, 16, 32, 64, 128]
    listToPredict = [256]
    benchmark_x = dictTmp[benchmark]["x"]
    benchmark_y = dictTmp[benchmark]["y"]
    index = benchmark_x.index(len(listToLearn))
    MAPE = benchmark_y[index]
    relativeCost = returnRelativeCost(benchmark=benchmark, variablesToLearn=listToLearn,
                                      variablesToPredict=listToPredict, fixedClassOrProcess="Class", fixed="C")
    dictRowData = {"ベンチマーク名": benchmark.upper(
    ), "平均絶対誤差率[％]": MAPE, "相対コスト[％]": relativeCost}
    iDF = pd.DataFrame.from_dict(dictRowData, orient='index').T
    tmpDF = tmpDF.append(iDF)
tmpDFMean = tmpDF.mean()
type(tmpDFMean)
print(tmpDF.to_latex(index=False))

In [ ]:
# dictTmp

plt.figure(figsize=(5.72, 4), dpi=200)
for benchmark in list(dictTmp.keys()):
    x = dictTmp[benchmark]["x"]
    y = dictTmp[benchmark]["y"]
    plt.plot(x, y, marker='o', label=benchmark.upper())
    plt.legend()
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("平均絶対誤差率[％]")

In [ ]:
plt.figure(figsize=(5.72, 4), dpi=200)

# Extra-PでfixProcessデータを入力して出力したモデルの図時
plot_x = np.linspace(0.8, 256, 500)
# -3590464.6990329633 + 3759195.349891038 * p^(1/4)
plot_y = []
for x in plot_x:
    plot_y.append(2286768.3333333326 + 301997.61904761934 * math.log2(x)**(1))
plt.plot(plot_x, plot_y, label="ExtraP")

x = [1, 2, 4, 8, 16, 32, 64, 128]
y = [1984770.0, 2263540.0, 2821070.0, 3936140.0,
     3936140.0, 3936140.0, 3936140.0, 3936140.0]
x = np.array(x).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)
plt.scatter(x, y, marker="o", label="予測に用いた関数コール回数")
plot_x = np.array(plot_x).reshape(-1, 1)
x_target = [256]
y_target = [3936140]
plt.scatter(x_target, y_target, marker="o", label="予測したい関数コール回数の実測値")

benchmarkName = "CG"
functionName = "ICNVRT"

# 線形モデル
# 対数モデル

# 反比例モデル
modelIpMk2 = ModelIp_mk2(train_x=x, train_y=y, target_x=x_target, target_y=y_target,
                         benchmark_name=benchmarkName, function_name=functionName)
modelIpMk2.calc_lr()
plot_y_IpMk2 = modelIpMk2.predict(plot_x)
plt.plot(plot_x, plot_y_IpMk2, label="反比例モデル")
# 線形飽和モデル
modelBranchMk2 = ModelBranch_mk2(train_x=x, train_y=y, target_x=x_target,
                                 target_y=y_target, benchmark_name=benchmarkName, function_name=functionName)
modelBranchMk2.calc_lr()
plot_y_BranchMk2 = modelBranchMk2.predict(plot_x)
plt.plot(plot_x, plot_y_BranchMk2, label="線形飽和モデル")
# # 線形モデル
# model_lin = ModelLin(x, y, "CG", "ICNVRT", test_ratio=0)
# model_lin.calc_lr()
# plot_y_lin = model_lin.predict(plot_x)
# plt.plot(plot_x, plot_y_lin, label="線形モデル")
# # 対数モデル
# model_log10 = ModelLog10(x, y, "CG", "ICNVRT", test_ratio=0)
# model_log10.calc_lr()
# plot_y_log10 = model_log10.predict(plot_x)
# plt.plot(plot_x, plot_y_log10, label="対数モデル")
# # 反比例モデル
# model_ip = ModelIP(x, y, "CG", "ICNVRT", test_ratio=0)
# model_ip.calc_lr()
# plot_y_ip = model_ip.predict(plot_x)
# plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# # 線形飽和モデル
# model_branch = ModelBranch(x, y, "CG", "ICNVRT", test_ratio=0)
# model_branch.calc_lr()
# plot_y_branch = model_branch.predict(plot_x)
# plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")

plt.scatter(x, y, marker="o")

In [ ]:
# 実際にプロットする


# print(f"fix={fix}, benchmarkClasses={benchmarkClasses}, fixedProcess={fixedProcess}, Processes={processes}, FixedBenchmarkClass={fixedBenchmarkClass}")
# print(f"targetNumOfProcess={targetNumOfProcess}, targetProblemSize={fixedBenchmarkClass}, fix={fix}")

# DF = returnRawDFperBenchmark(Benchmark="mg", fix="Process", benchmarkClass=["A", "B", "C", "D"], Processes=[
#                              1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="B", FixedProcess=64)
# DF.dropna(how='any')
# DF

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

In [ ]:
benchmarkNamesExcludeBTSP = ["cg", "ep", "ft", "is", "lu", "mg"]
# classes = ["A", "B", "C", "D"]
classes = ["B"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
targetIndex = -1
csvDirPath = "./csv_files/"

dfByDatumExcludeBTSP = returnDFSummarizedData(
    benchmarkNames=benchmarkNamesExcludeBTSP, classes=classes, processes=processes, targetIndex=targetIndex, csvDirPath=csvDirPath)
# dfByDatumExcludeBTSP

dictForLatexTable = {}
numOfData = 0
for benchmarkName in benchmarkNamesExcludeBTSP:
    dictForLatexTable[benchmarkName] = dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName]
    numOfData += len(
        dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName])

numOfData

In [ ]:


listForDF = []

for benchmarkName in benchmarkNamesExcludeBTSP:
    listForDF.append(returnSeriesOfDatumPerBenchmark(
        inputDF=dictForLatexTable[benchmarkName]))
DF = pd.DataFrame(listForDF)
print(DF.to_latex(index=False))

In [ ]:
test_returnSeriesOfDatumPerBenchmark()

In [ ]:
resultIs = dictForLatexTable["is"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
datumX = resultIsAtModelBranch["usedDataX"].tolist()
datumY = resultIsAtModelBranch["usedDataY"].tolist()

datumX
datumY

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
resultIs = dictForLatexTable["ft"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
resultIsAtModelBranchOfNotLowMAPE = resultIsAtModelBranch[
    resultIsAtModelBranch["MAPEOfBestModel"] > 1]
resultIsAtModelBranchOfNotLowMAPE
datumX = resultIsAtModelBranchOfNotLowMAPE["usedDataX"].tolist()
datumY = resultIsAtModelBranchOfNotLowMAPE["usedDataY"].tolist()

datumX
datumY

for dataIndex in range(len(datumX)):
    plt.figure()
    plt.scatter(datumX[dataIndex], datumY[dataIndex])

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
# 生データの取得
cgDF = returnCollectedExistingData(benchmarkNames=["cg"], classes=["A", "B", "C", "D"], processes=[
                                   1, 2, 4, 8, 16, 32, 64, 128, 256], csvDirPath="./csv_files/")
cgDF
# ベンチマーククラスがAの情報を取得
cgDFfixedA = cgDF[cgDF["benchmarkClass"] == "A"]
cgDFfixedA
# 関数名のリストを取得
functionNames = sorted(list(set(cgDFfixedA["functionName"])))
print(functionNames)

# 関数名を関数名のリストから抽出
functionNameCG = cgDFfixedA[cgDFfixedA["functionName"] == "CG"]
functionNameCG

# 説明変数と目的変数とをリスト化したものを抽出
# プロセス数
raw_x = functionNameCG['process'].tolist()
# 関数コール回数
raw_y = functionNameCG['functionCallNum'].tolist()

print(f"raw_x={raw_x}")
print(f"raw_y={raw_y}")

bencmarkName = "CG"
functionName = "CG"
fixProcessOrClass = "Class"
fixed = "A"
targetProcess = 256
targetBenchmarkClass = fixed
targetFunctionCallNum = raw_y[-1]
returnSeriesOfData(benchmarkName="benhmarkName", functionName="functionName", rawX=[1, 2, 3], rawY=[
                   1, 2, 3], fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files/")

In [ ]:
%reset

In [ ]:
# ノートブック中で変数のみを記述することでデータフレームをきれいに表示させる設定の有効化
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 問題サイズD, コア数256での関数コール回数を予測する

In [ ]:
# TODO：BT, SP以外のベンチマーク名を入れる
benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']
classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dictForSummarizedResult = {}
columnName = ["benchmarkName", "functionName", "score", "relativeErrorRate"]
dfForSummarizedResult = pd.DataFrame(columns=columnName)
for benchmarkName in benchmarkNames:
    # ベンチマークごとにscoreを保持するためのリスト
    listForSummarizedResultPerBenchmarkName = []
    # 学習用生データ
    DF = returnCollectedExistingData(benchmarkNames=[
                                     benchmarkName], classes=classes, processes=processes, csvDirPath="./csv_files/")
    # 重複のない関数名のリスト
    functionNames = list(set(DF["functionName"]))
    usefulFunctionNames = []
    # このループで関数ごとのデータが問題サイズパターン数xコア数パターン数 分だけ存在する関数名のリストを作成する
    for functionName in functionNames:
        # 関数ごとに生データを集計
        dfPerFunction = DF[DF["functionName"] == functionName]
        if len(classes) * len(processes) == len(dfPerFunction):
            usefulFunctionNames.append(functionName)
    if len(usefulFunctionNames) == 0:
        continue
    # 関数ごとのデータを抽出
    for functionName in usefulFunctionNames:
        # 問題サイズを数値化したカラムを追加
        listBenchmarkClass = DF["benchmarkClass"].tolist()
        DFWithNumInBenchmarkClass = DF.assign(
            benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))
        # 学習用データ
        dfPerFunctionForTrain = DFWithNumInBenchmarkClass[(
            DFWithNumInBenchmarkClass["functionName"] == functionName)]
        dfPerFunctionForTest = DFWithNumInBenchmarkClass[(DFWithNumInBenchmarkClass["functionName"] == functionName) & (
            DFWithNumInBenchmarkClass["benchmarkClass"] == "D") & (DFWithNumInBenchmarkClass["process"] == 256)]

        # x:説明変数, t:目的変数
        trainX = dfPerFunctionForTrain[["process", "benchmarkClassInNum"]]
        trainT = dfPerFunctionForTrain[["functionCallNum"]]
        testX = dfPerFunctionForTest[["process", "benchmarkClassInNum"]]
        testT = dfPerFunctionForTest[["functionCallNum"]]
        # 重回帰分析する
        reg_model = LinearRegression()
        reg_model.fit(trainX, trainT)
        # 関数ごとの結果をベンチマークごとの結果に入れる
        scorePerFunction = reg_model.score(trainX, trainT)
        listForSummarizedResultPerBenchmarkName.append(scorePerFunction)
        # 予測を実施して、相対誤差を算出
        predictedTByTestX = reg_model.predict(testX)
        predictedData = predictedTByTestX[0][0]
        realData = testT["functionCallNum"].tolist()[0]
        relativeErrorPerFunction = abs(predictedData - realData)/realData * 100
        ##
        dfPerFunction = pd.DataFrame(index=columnName, data=[
                                     benchmarkName, functionName, scorePerFunction, relativeErrorPerFunction]).T
        dfForSummarizedResult = dfForSummarizedResult.append(dfPerFunction)

# ( A ~ D ) * (1 ~ 256) のすべての条件を
# 満たしていたら、リストに追加
# 満たしていなければ、なにもしない

In [ ]:
dfForSummarizedResult

In [ ]:
inputDF = dfForSummarizedResult


benchmarkNamesInDF = list(set(dfForSummarizedResult["benchmarkName"].tolist()))

listForLatexTable = []
for benchmarkName in benchmarkNamesInDF:
    print(benchmarkName)
    inputDFPerBenchmark = inputDF[inputDF["benchmarkName"] == benchmarkName]
    meanData = inputDFPerBenchmark.mean()
    print(type(meanData))
    meanData["benchmarkName"] = f"{benchmarkName.upper()}({len(inputDFPerBenchmark)})"
    listForLatexTable.append(meanData)
DF = pd.DataFrame(listForLatexTable)

DF = DF.sort_index(axis='columns')
DF
# relativeErrorの単位は[%]ではない。scoreの値はscore()で取得できたもの
DF.columns = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)", "決定係数"]
print(DF.to_latex(index=False))

In [ ]:
forInputDF = returnDFSummarizedData(
    benchmarkNames=["cg", "ep", "ft", "is", "lu", "mg"],
    classes=["C"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    targetIndex=-1,
    csvDirPath="./csv_files/",
)

benchmarkNames = list(set(forInputDF["benchmarkName"].tolist()))
columnsNames = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)"]
listForRelativeErrorTable = []
for benchmarkName in benchmarkNames:
    forInputDFPerBenchmark = forInputDF[forInputDF["benchmarkName"]
                                        == benchmarkName]
    column1 = f"{benchmarkName.upper()}({len(forInputDFPerBenchmark)})"
    seriesOfMean = forInputDFPerBenchmark.mean()
    seriesOfMeanRelativeErrorRate = seriesOfMean["RelativeErrorRate"]
    column2 = int(seriesOfMeanRelativeErrorRate * 100) / 100
    listForRelativeErrorTable.append([{column1}, {column2}])
print(pd.DataFrame(listForRelativeErrorTable,
      columns=columnsNames).to_latex(index=False))

# forInputDFPerBenchmark

In [ ]:
pd.DataFrame(listForRelativeErrorTable, columns=columnsNames)

In [8]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb


# 引数として渡されたDFから、関数ごとに「関数名 | ベンチマーク名 | 説明変数 | 目的変数 | 集計結果」を保持したDFを作成する関数
# 引数として渡されたDFにはベンチマーク・関数はそれぞれ１種類のデータが格納されている
def returnDFtoMakeSummary(inputDF, benchmarkName="benchmarkName", validFunctionName="validFunctionName", targetClass="D", targetProcess=256, expVarColNames=[], resVarColNames=[]):
    # モデルを一括で作成
    targetDF = inputDF[(inputDF["benchmarkClass"]==targetClass) & (inputDF["process"]==targetProcess)]
    dropIndex = inputDF.index[(inputDF["benchmarkClass"]==targetClass) | (inputDF["process"]==targetProcess)]
    droppedInputDF = inputDF.drop(dropIndex)
    models = Models(inputDF=droppedInputDF, expVarColNames=expVarColNames, resVarColNames=resVarColNames, targetDF=targetDF)
    # 学習
    models.setUpDataBeforeCalcLr()
    models.calcLr()
    # MAPE・相対誤差率を算出
    models.calcMAPE()
    models.calcRelativeErrorRate()
    # 結果の格納
    dictAggregateResult = {"MAPE":models.returnCalculatedMAPE(), "relativeErrorRate":models.returnRelativeErrorRateDict()}
    expVarDict = models.returnExpVarDatumDF().to_dict(orient='list')
    resVarDict = models.returnResVarDatumDF().to_dict(orient='list')
    modelsName = models.returnModelsName()
    dictDatumForDF = {"functionName":validFunctionName, "benchmarkName":benchmarkName, "expVarDatumDict":expVarDict, "resVarDatumDict":resVarDict, "modelsName":modelsName, "dictAggregateResult":dictAggregateResult}
    listDatumKeysForDF = dictDatumForDF.keys()
    listDatumValuesForDF = dictDatumForDF.values()
    returnDF = pd.DataFrame(index=listDatumKeysForDF, data=listDatumValuesForDF).T
    pass


benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']
benchmarkName = 'cg'

classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
targetClass = classes[-1]
targetProcess = processes[-1]

# 学習用生データ
DF = returnCollectedExistingData(benchmarkNames=[
                                 benchmarkName], classes=classes, processes=processes, csvDirPath="./csv_files/")
DFByValidFunction = returnDFwithFunctionsExecUnderAllConditions(
    inputDF=DF, classes=classes, processes=processes)
# 問題サイズを数値化したカラムを追加
listBenchmarkClass = DFByValidFunction["benchmarkClass"].tolist()
DFWithNumInBenchmarkClass = DFByValidFunction.assign(
    benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))

# すべての条件で実行された関数名のリスト
validFunctionNames = list(
    set(DFWithNumInBenchmarkClass["functionName"].tolist()))
# データ内にあるベンチマーク名のリスト
benchmarkNames = set(DFWithNumInBenchmarkClass["benchmarkName"].tolist())

# 説明変数のカラム名のリスト
expVarColNames = ["process", "benchmarkClassInNum"]
# 目的変数のカラム名のリスト
resVarColNames = ["functionCallNum"]

for benchmarkName in benchmarkNames:
    for validFunctionName in validFunctionNames:
        # 3モデルを一気に作成するmodels()を利用
        inputDFperFunction = DFWithNumInBenchmarkClass[(DFWithNumInBenchmarkClass["functionName"] == validFunctionName) & (DFWithNumInBenchmarkClass["benchmarkName"] == benchmarkName)].reset_index()
        targetDFperFunction = inputDFperFunction[(inputDFperFunction["benchmarkClass"]==targetClass) & (inputDFperFunction["process"]==targetProcess)]
        # inputDFperFunctionの中で条件に当てはまるデータを削除
        dropIndex = inputDFperFunction.index[(inputDFperFunction["benchmarkClass"] == targetClass) | (inputDFperFunction["process"]==targetProcess)]
        inputDFperFunction = inputDFperFunction.drop(dropIndex)
        modelsPerFunction = Models(inputDF=inputDFperFunction, expVarColNames=expVarColNames, resVarColNames=resVarColNames, targetDF=targetDFperFunction)
        modelsPerFunction.setUpDataBeforeCalcLr()
        modelsPerFunction.calcLr()
        # 学習データに対するMAPEを算出し、変数に結果を保持した辞書を格納
        modelsPerFunction.calcMAPE()
        dictCalcedMAPE = modelsPerFunction.returnCalculatedMAPE()
        # 予測対象データに対する相対誤差率を算出し、変数に結果を保持した辞書を格納
        modelsPerFunction.calcRelativeErrorRate()
        dictRelativeErrorRate = modelsPerFunction.returnRelativeErrorRateDict()
        
        # 関数ごとに「関数名 | ベンチマーク名 | 説明変数 | 目的変数 | 集計結果」を保持したDFを作成
        expVarDatumDF = modelsPerFunction.returnExpVarDatumDF()
        resVarDatumDF = modelsPerFunction.returnResVarDatumDF()
        # 説明変数のDFを辞書にする
        expVarDatumDict = expVarDatumDF.to_dict(orient='list')
        # 目的変数のDFを辞書にする
        resVarDatumDict = resVarDatumDF.to_dict(orient='list')
        modelsName = modelsPerFunction.returnModelNames()
        # 集計結果（MAPE、相対誤差率）
        dictAggregateResult = {"MAPE":dictCalcedMAPE, "relativeErrorRate":dictRelativeErrorRate}
        dictDatumForDF = {"functionName":validFunctionName, "benchmarkName":benchmarkName, "expVarDatumDict":expVarDatumDict, "resVarDatumDict":resVarDatumDict, "modelsName":modelsName, "dictAggregateResult":dictAggregateResult}
        listDatumKeysForDF = dictDatumForDF.keys()
        listDatumValuesForDF = dictDatumForDF.values()
        DFperValidFunction = pd.DataFrame(data=listDatumValuesForDF, index=listDatumKeysForDF).T

        


def test_returnDFtoMakeSummary():
    # 入力用DF、inputDFを作成する
    plotX = np.linspace(1, 20, 10)
    plotY = np.linspace(21, 40, 10) 
    # functionCallNum
    functionCallNumLin = plotX + 2*plotY + 4
    functionCallNumIp  = 1/plotX + 2/plotY  + 4
    functionCallNumLog = np.log10(plotX) + 2*np.log10(plotY) + 4
    # processes
    process = np.linspace(1, 20, 10)
    # benchmarkClassInNum
    benchmarkClassInNum = np.linspace(21, 40, 10)
    # functionName
    functionNameLin = "functionNameLin"
    functionNameIp  = "functionNameIp"
    functionNameLog = "functionNameLog"
    # benchmarkName
    benchmarkNameLin = "benchmarkNameLin"
    benchmarkNameIp = "benchmarkNameIp"
    benchmarkNameLog = "benchmarkNameLog"
    # benchmarkClass
    benchmarkClass = ["Z"]*len(benchmarkClassInNum)
    benchmarkClass[-1] = "X"
    
    dictForDFatLin = {"functionCallNum":functionCallNumLin, "process":process, "benchmarkClassInNum":benchmarkClassInNum, "benchmarkClass":benchmarkClass}
    inputDFatLin = pd.DataFrame(dictForDFatLin)
    inputDFatLin["functionName"] = functionNameLin
    inputDFatLin["benchmarkName"] = benchmarkNameLin
    
    dictForDFatIp  = {"functionCallNum":functionCallNumLin, "process":process, "benchmarkClassInNum":benchmarkClassInNum, "benchmarkClass":benchmarkClass}
    inputDFatIp  = pd.DataFrame(dictForDFatIp)
    inputDFatIp["functionName"] = functionNameIp
    inputDFatIp["benchmarkName"] = benchmarkNameIp
    
    dictForDFatLog = {"functionCallNum":functionCallNumLin, "process":process, "benchmarkClassInNum":benchmarkClassInNum, "benchmarkClass":benchmarkClass}
    inputDFatLog = pd.DataFrame(dictForDFatLog)
    inputDFatLog["functionName"] = functionNameLog
    inputDFatLog["benchmarkName"] = benchmarkNameLog
    
    # 関数の実行に必要な引数を作成する
    targetClass = benchmarkClass[-1]
    targetProcess = processes[-1]
    
    # returnDFtoMakeSummary()の実行
    resultAtLin = returnDFtoMakeSummary(inputDF = inputDFatLin, targetClass=targetClass, targetProcess=targetProcess)
    resultAtIp  = returnDFtoMakeSummary(inputDF = inputDFatIp,  targetClass=targetClass, targetProcess=targetProcess)
    resultAtLog = returnDFtoMakeSummary(inputDF = inputDFatLog, targetClass=targetClass, targetProcess=targetProcess)
    
    # linについて
    assert len(resultAtLin) == 1
    # functionName
    assert functionNameAtLin[0] == functionNameLin
    # benchmarkName
    assert benchmarkNameAtLinResult[0] == benchmarkNameLin
    # expVarDatumDict
    expVarDictLinResult = resultAtLin.at[resultAtLin.index[0], "expVarDatumDict"]
    processLinResult = expVarDictLinResult["processes"]
    benchmarkClassInNumLinResult = expVarDictLinResult["benchmarkClassInNum"]
    assert processLinResult == processes
    assert benchmarClassInNumLinResult == benchmarkClassInNum
    # resVarDatumDict
    resVarDictLinResult = resultAtLin.at[resultAtLin.index[0], "resVarDatumDict"]
    functionCallNumLinResult = resVarDictLinResult["functionCallNum"]
    assert functionCallNumLinResult == functionCallNumLin
    # modelsName
    modelsNameLinResult = resultAtLin.at[resultAtLin.index[0], "modelsName"]
    assert "modelLin" in modelsNameLinResult
    assert "modelIp" in modelsNameLinResult
    assert "modelLog" in modelsNameLinResult
    # dictAggregateResult
    dictAggregateResult = resultAtlin.at[resultAtLin.index[0], "dictAggregateResult"]
    MAPELinResult = dictAggregateResult["MAPE"]
    assert MapeLinResult["modelLin"] < 1.0
    relativeErrorRateLinResult = dictAggregateResult["MAPE"]
    assert relativeErrorRateLinResult["modelLin"] < 1.0
    
    # Ipについて
    assert len(resultAtIp) == 1
    # functionName
    assert functionNameAtIp[0] == functionNameIp
    # benchmarkName
    assert benchmarkNameAtIpResult[0] == benchmarkNameIp
    # expVarDatumDict
    expVarDictIpResult = resultAtIp.at[resultAtIp.index[0], "expVarDatumDict"]
    processIpResult = expVarDictIpResult["processes"]
    benchmarkClassInNumIpResult = expVarDictIpResult["benchmarkClassInNum"]
    assert processIpResult == processes
    assert benchmarClassInNumIpResult == benchmarkClassInNum
    # resVarDatumDict
    resVarDictIpResult = resultAtIp.at[resultAtIp.index[0], "resVarDatumDict"]
    functionCallNumIpResult = resVarDictIpResult["functionCallNum"]
    assert functionCallNumIpResult == functionCallNumIp
    # modelsName
    modelsNameIpResult = resultAtIp.at[resultAtIp.index[0], "modelsName"]
    assert "modelLin" in modelsNameIpResult
    assert "modelIp"  in modelsNameIpResult
    assert "modelLog" in modelsNameIpResult
    # dictAggregateResult
    dictAggregateResult = resultAtIp.at[resultAtIp.index[0], "dictAggregateResult"]
    MAPEIpResult = dictAggregateResult["MAPE"]
    assert MapeIpResult["modelIp"] < 1.0
    relativeErrorRateIpResult = dictAggregateResult["MAPE"]
    assert relativeErrorRateIpResult["modelIp"] < 1.0
    
    # Logについて
    assert len(resultAtLog) == 1
    # functionName
    assert functionNameAtLog[0] == functionNameLog
    # benchmarkName
    assert benchmarkNameAtLogResult[0] == benchmarkNameLog
    # expVarDatumDict
    expVarDictLogResult = resultAtLog.at[resultAtLog.index[0], "expVarDatumDict"]
    processLogResult = expVarDictLogResult["processes"]
    benchmarkClassInNumLogResult = expVarDictLogResult["benchmarkClassInNum"]
    assert processLogResult == processes
    assert benchmarClassInNumLogResult == benchmarkClassInNum
    # resVarDatumDict
    resVarDictLogResult = resultAtLog.at[resultAtLog.index[0], "resVarDatumDict"]
    functionCallNumIpResult = resVarDictLogResult["functionCallNum"]
    assert functionCallNumLogResult == functionCallNumLog
    # modelsName
    modelsNameLogResult = resultAtLog.at[resultAtLog.index[0], "modelsName"]
    assert "modelLin" in modelsNameLogResult
    assert "modelIp"  in modelsNameLogResult
    assert "modelLog" in modelsNameLogResult
    # dictAggregateResult
    dictAggregateResult = resultAtlog.at[resultAtLog.index[0], "dictAggregateResult"]
    MAPELogResult = dictAggregateResult["MAPE"]
    assert MapeLogResult["modelLog"] < 1.0
    relativeErrorRateLogResult = dictAggregateResult["MAPE"]
    assert relativeErrorRateLogResult["modelLog"] < 1.0
    
test_returnDFtoMakeSummary()

ValueError: at least one array or dtype is required

In [2]:
DFperValidFunction

,functionName,benchmarkName,expVarDatumDict,resVarDatumDict,modelsName,dictAggregateResult
0,CONJ_GRAD,cg,"{'process': [2, 4, 8, 16, 32, 64, 128, 2, 4, 8...","{'functionCallNum': [16.0, 16.0, 16.0, 16.0, 1...","[modelLin, modelIp, modelLog]","{'MAPE': {'modelLin': [51.0005706761659], 'mod..."


In [3]:
DF

,functionName,functionCallNum,benchmarkName,benchmarkClass,process
0,.TAU_application,1.0,cg,A,2
1,CG,1.0,cg,A,2
2,MPI_Finalize(),1.0,cg,A,2
3,INITIALIZE_MPI,1.0,cg,A,2
4,MPI_Init(),1.0,cg,A,2
...,...,...,...,...,...
17,MPI_Reduce(),1.0,cg,D,256
18,SETUP_SUBMATRIX_INFO,1.0,cg,D,256
19,MPI_Comm_size(),1.0,cg,D,256
20,MPI_Comm_rank(),1.0,cg,D,256


In [5]:
{"functionCallNum":functionCallNumLin, "processes":processes, "benchmarkClassInNum":benchmarkClassInNum, "benchmarkClass":benchmarkClass}

NameError: name 'functionCallNumLin' is not defined